# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
data_file = os.path.join("..","WeatherPy","exports","CityList.csv")
work_df = pd.read_csv(data_file)
work_df = work_df.drop(['Unnamed: 0'], axis=1)
work_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ribeira Grande,38.52,-28.70,62.600,67,20,2.10,PT,1600323578
1,Bluff,-46.60,168.33,46.004,70,88,6.26,NZ,1600323328
2,Mataura,-46.19,168.86,46.004,70,100,6.26,NZ,1600323578
3,Nikolskoye,59.70,30.79,51.800,76,75,3.00,RU,1600323579
4,Pevek,69.70,170.31,35.348,96,100,6.25,RU,1600323579


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
import gmaps.datasets
gmaps.configure(g_key)

In [13]:
locations = work_df[['Lat', 'Lng']]
weights = work_df['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
work_df_filtered = work_df[work_df["Max Temp"] < 80] #degrees are in Kelvin, need to go back and fix. 300 Kelvin is close to 80 degrees F
work_df_filtered = work_df_filtered[work_df_filtered["Max Temp"] > 70] #degrees are in Kelvin, need to go back and fix. 294 Kelvin is close to 70 degrees F
work_df_filtered = work_df_filtered[work_df_filtered["Wind Speed"]<= 15]
work_df_filtered = work_df_filtered[work_df_filtered["Humidity"]<= 60]
work_df_filtered.shape



(21, 9)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
locations_lat = work_df_filtered['Lat'].to_list()
locations_lng = work_df_filtered['Lng'].to_list()

In [19]:
result_lat = []
result_lng = []
result_name = []

for i in range(0,len(locations_lat)):
#for i in range(0,1):
    # geocoordinates for the base of our search
    target_location = str(f"{locations_lat[i]},{locations_lng[i]}") # coordinates
#target_location = "-28.78,32.04"
# distance, IN METERS, within which the place results must live
    target_radius = 5000

# type of establishment that we'd like to use to filter our place results
    target_type = "hotel"

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up a dictionary to store all query parameters
    params = {
        "location": target_location,
#    "keyword": target_keyword,
        "radius": target_radius,
#     "rankby": "distance",
        "type": target_type,
        "key": g_key
}

# run a request using our params dictionary
    response = requests.get(base_url, params=params)
    response_json = response.json()
    result_lat.append(response_json['results'][0]['geometry']["location"]["lat"])
    result_lng.append(response_json['results'][0]['geometry']["location"]["lng"])
    result_name.append(response_json['results'][0]['name'])

In [20]:
result_coord = zip(result_lat,result_lng)
result_coord = list(result_coord)
#result_coord

In [21]:
marker_locations = result_coord

#fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [102]:
#I can't get any of this to work

#keys = ['hotel name', 'location']
#values = [str(result_name),str(result_coord)]
#tag_dict = dict(zip(keys, values))


#hotel_locations = [row['location'] for row in tag_dict]
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap

#cannot see City or Country as an output in the API
#info_box_template = """
#<dl>
#<dt>Name</dt><dd>{Hotel Name}</dd>
#<dt>City</dt><dd></dd></dd>
#<dt>Country</dt><dd></dd></dd>
#</dl>
#"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
    #plant_info = [info_box_template.format(**plant) for plant in nuclear_power_plants]
#hotel_info = [info_box_template.format(**row) for row in hotel_locations]
#locations = work_df_filtered[["Lat", "Lng"]]
#fig = gmaps.figure()
#fig.add_layer(marker_layer)
#fig

In [ ]:
# Add marker layer ontop of heat map


# Display figure
